In [3]:
import os
import gc
import shutil
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from scipy.optimize import curve_fit
#from sklearn.metrics import r2_score
from ccmodels.preprocessing.connectomics import client_version, subset_v1l234
from ccmodels.preprocessing.selectivity import orientation_extractor,orientation_extractor_v2, von_mises, von_mises_single, is_selective, cell_area_identifiers, fpd_assignment, identify_multiscan


Connecting microns@db.datajoint.com:3306


In [4]:
#Identify desired neurons, in this case v1 neurons from L234
client = client_version(661)
area_v1_neurons = cell_area_identifiers('V1')
v1l234_neur = subset_v1l234(client, table_name = 'coregistration_manual_v3', area_df = area_v1_neurons)
v1l234_neur = v1l234_neur[v1l234_neur['pt_root_id'] != 0]

#Identify neurons that have been scaned multiple vs one time only
multiscan_ids = identify_multiscan(v1l234_neur)
v1l234_singlescan = v1l234_neur[~v1l234_neur['pt_root_id'].isin(multiscan_ids)]
v1l234_multiscan = v1l234_neur[v1l234_neur['pt_root_id'].isin(multiscan_ids)]

Using deprecated pyarrow serialization method, please upgrade CAVEClient>=5.9.0 with pip install --upgrade caveclient


In [5]:
v1l234_singlescan[(v1l234_singlescan['session'] == 7)&(v1l234_singlescan['scan_idx'] == 4)]

,pt_root_id,id,session,scan_idx,unit_id,pt_position,brain_area,pial_distances,cortex_layer
0,864691135614842827,3045,7,4,9575,"[136400, 170640, 17951]",V1,"[484.03480350901486, 330.8437941623397, 718.04...",L4
14,864691136619416973,10848,7,4,2673,"[165328, 114000, 22174]",V1,"[619.0524894915164, 115.2308886674908, 886.960...",L2/3
34,864691134940425315,8946,7,4,9229,"[269920, 159344, 20254]",V1,"[1020.0205129461647, 332.37987201043154, 810.1...",L4
89,864691136108732088,5379,7,4,5615,"[129168, 135152, 21555]",V1,"[467.5888152771324, 186.91072305261594, 862.20...",L2/3
91,864691136195052364,1337,7,4,5672,"[129344, 136352, 20478]",V1,"[467.8717887794002, 191.75381524635065, 819.12...",L2/3
...,...,...,...,...,...,...,...,...,...
9137,864691135940066561,7544,7,4,5537,"[160176, 134704, 20300]",V1,"[591.3050191618516, 195.9356432381131, 812.000...",L2/3
9159,864691135610499847,11641,7,4,9058,"[140176, 165920, 15919]",V1,"[500.7268286520684, 313.3520386008282, 636.760...",L4
9162,864691135304303015,5275,7,4,5505,"[171888, 132064, 21298]",V1,"[638.895113021469, 189.49889946250647, 851.920...",L2/3
9207,864691135469349970,3266,7,4,8968,"[184320, 162832, 24098]",V1,"[677.7074513967357, 316.43665412140945, 963.92...",L4


In [6]:
fpd = fpd_assignment(7,4)

In [7]:
unit_key = {'session':7, 'scan_idx':4, 'unit_id':9575}
unit_key2 = {'session':7, 'scan_idx':4, 'unit_id':2673}

In [8]:
dft = orientation_extractor_v2(unit_key, fpd)
dft2 = orientation_extractor_v2(unit_key2, fpd)

In [12]:
pd.concat([dft, dft2], axis = 0)

,orientation,mean_activity,trial_id,radians,session,scan_idx,unit_id
0,0.0,2.720659e+00,0,0.000000,7,4,9575
1,157.5,4.504594e+00,0,2.748894,7,4,9575
2,135.0,3.592107e-09,0,2.356194,7,4,9575
3,337.5,1.458118e+00,0,5.890486,7,4,9575
4,270.0,1.723097e-08,0,4.712389,7,4,9575
...,...,...,...,...,...,...,...
635,292.5,7.271685e-10,39,5.105088,7,4,2673
636,247.5,7.331164e-10,39,4.319690,7,4,2673
637,45.0,7.365958e-10,39,0.785398,7,4,2673
638,157.5,7.616177e-10,39,2.748894,7,4,2673


In [10]:
dft

,orientation,mean_activity,trial_id,radians,session,scan_idx,unit_id
0,0.0,2.720659e+00,0,0.000000,7,4,9575
1,157.5,4.504594e+00,0,2.748894,7,4,9575
2,135.0,3.592107e-09,0,2.356194,7,4,9575
3,337.5,1.458118e+00,0,5.890486,7,4,9575
4,270.0,1.723097e-08,0,4.712389,7,4,9575
...,...,...,...,...,...,...,...
635,292.5,2.167453e-09,39,5.105088,7,4,9575
636,247.5,2.124843e-09,39,4.319690,7,4,9575
637,45.0,2.180766e-09,39,0.785398,7,4,9575
638,157.5,2.137942e-09,39,2.748894,7,4,9575


In [3]:
#Identify desired neurons, in this case v1 neurons from L234
client = client_version(661)
area_v1_neurons = cell_area_identifiers('V1')
v1l234_neur = subset_v1l234(client, table_name = 'coregistration_manual_v3', area_df = area_v1_neurons)
v1l234_neur = v1l234_neur[v1l234_neur['pt_root_id'] != 0]

Using deprecated pyarrow serialization method, please upgrade CAVEClient>=5.9.0 with pip install --upgrade caveclient


In [4]:
multiscan_ids = identify_multiscan(v1l234_neur)

In [5]:
v1l234_singlescan = v1l234_neur[~v1l234_neur['pt_root_id'].isin(multiscan_ids)]
v1l234_multiscan = v1l234_neur[v1l234_neur['pt_root_id'].isin(multiscan_ids)]
v1l234_singlescan.shape, v1l234_multiscan.shape, v1l234_neur.shape

((5067, 9), (2025, 9), (7092, 9))

In [6]:
#Container with saved data
data = []
            
#Von Mises
columns = ['root_id', 'session', 'scan_idx','cell_id', 'pvalue','tuning_type', 
        'r_squared_diff', 'mean_r_sqrd', 'A', 'phi', 'k', 'activity', 'orientations']

for mult_id in tqdm(multiscan_ids, desc = f'Extracting multiscan neurons data') :
    subset_multiscan = v1l234_multiscan[v1l234_multiscan['pt_root_id'] == mult_id]
    scan_acts = []
    
    for row in range(subset_multiscan.shape[0]):
        unit_key = {'session':subset_multiscan['session'].values[row], 'scan_idx':subset_multiscan['scan_idx'].values[row], 'unit_id':subset_multiscan['unit_id'].values[row]}
        fpd = fpd_assignment(subset_multiscan['session'].values[row], subset_multiscan['scan_idx'].values[row])
        df = orientation_extractor(unit_key, fpd)
        scan_acts.append(df)
    
    all_acts = pd.concat(scan_acts, axis = 1)

    #Calculating max activity for constraining von mises, for plotting data and fit
    gp = df.groupby('radians').mean().reset_index()
    max_act_rad = gp.sort_values('mean_activity', ascending = False)['radians'].values[0]
            
    #Using Von Mises
    pars_d, pcov_d = curve_fit(lambda theta, A, k: von_mises(theta, A, max_act_rad, k), df['radians'], df['mean_activity'],
                                bounds = ([-np.inf, 0.1],[np.inf, np.inf]),
                                method = 'trf')

    pars_s, pcov_s = curve_fit(lambda theta, A, k: von_mises(theta, A, max_act_rad, k), df['radians'], df['mean_activity'],
                                bounds = ([-np.inf, 0.1],[np.inf, np.inf]),
                                method = 'trf')



    # Von Mises activities
    ate_d= [von_mises(i,pars_d[0], max_act_rad,  pars_d[1]) for i in gp['radians']]
    ate_s= [von_mises_single(i,pars_s[0], max_act_rad,  pars_s[1]) for i in gp['radians']]
            
    r2d = r2_score(gp['mean_activity'], ate_d)
    r2s = r2_score(gp['mean_activity'], ate_s)

    #Calculate difference in r2 score between the two model fist to identify fringe cases
    rdiff = r2d-r2s
            
    #single
    vs, ps, max_rad = is_selective(df, max_act_rad)


    #double
    vdb, pdb, max_rad = is_selective(df, max_act_rad, single = False)
            
            
    #save data
    data.append([mult_id, unit_key['session'], unit_key['scan_idx'], unit_key['unit_id'],ps,'single', rdiff, r2s, 
                        pars_d[0], max_act_rad,  pars_d[1], np.array(gp['mean_activity']), np.array(gp['radians'])])
            
    #save data
    data.append([mult_id, unit_key['session'], unit_key['scan_idx'], unit_key['unit_id'],pdb,'double', rdiff,r2d, 
                        pars_s[0], max_act_rad,  pars_s[1], np.array(gp['mean_activity']), np.array(gp['radians'])])
            
#Save the data
data_df = pd.DataFrame(data, columns = columns)
data_df.to_pickle(f'../data/in_processing/orientation_fits/multiscan_fits.pkl')


Extracting multiscan neurons data:   0%|          | 0/967 [00:00<?, ?it/s]

OSError: Cannot save file into a non-existent directory: '../data/in_processing/orientation_fits'

In [18]:
t = pd.read_pickle('/workspaces/microns_phase3_nda/data/in_processing/orientation_fits.pkl')

In [11]:
nda.Scan().fetch(format = 'frame')

nframes  nfields      fps
session scan_idx                           
4       7           40000        8  6.30090
        9           35112        8  6.30090
        10          40000        8  6.30090
5       3           40000        8  6.30090
        6           40000        8  6.30090
        7           40000        8  6.30090
6       2           40000        8  6.30090
        4           40000        8  6.30090
        6           40000        8  6.30090
        7           40000        8  6.30090
7       3           40000        8  6.30090
        4           40000        8  6.30090
        5           40000        8  6.30090
8       5           40000        8  6.30090
        7           40000        8  6.30090
        9           40000        8  6.30090
9       3           50000        6  8.61754
        4           50000        6  8.61754
        6           57000        4  9.62246

In [15]:
ors_all = pd.read_pickle(f"../data/in_processing/orientation_fits/{os.listdir('../data/in_processing/orientation_fits')[0]}")

#Loading the rest iteratively and concatenating
for file in tqdm(os.listdir('../data/in_processing/orientation_fits/')[1:], desc='Aggregating session, scan files'):
    if file!='.DS_Store':
        cont_df = pd.read_pickle(f'../data/in_processing/orientation_fits/{file}')
        ors_all = pd.concat([ors_all, cont_df], axis = 0)
    else:
        continue

#Saving the data
ors_all.to_pickle('../data/in_processing/orientation_fits.pkl')

Aggregating session, scan files:   0%|          | 0/15 [00:00<?, ?it/s]

In [16]:
shutil.rmtree('../data/in_processing/orientation_fits')